In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib.request import urlopen
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [27]:
url_skeleton = 'https://www.allsides.com/search?search={}&item_bundle=1&sort_by=node_created'

https://www.allsides.com/search?search=abortion&item_bundle=1&sort_by=node_created


**Importing the Modules**
- requests
- BeautifulSoup
- csv 
- re
- socket

In [28]:
import requests
from bs4 import BeautifulSoup
import csv
import re 

In [11]:
topics = ['abortion', 'defense-and-security', 'economy-and-jobs', 'energy', 'environment', \
          'foreign-policy', 'sustainability', 'gun-control-and-gun-rights', 'violence-america', \
            'healthcare', 'public-health', 'housing-and-homelessness', 'race', 'civil-rights']

In [ ]:
def soup_basics(item):
    page = requests.get(item)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

In [ ]:
url = "https://www.allsides.com/search?search=abortion&item_bundle=1&sort_by=node_created"

In [89]:
selectors = {
    'apnews.com': 'RichTextStoryBody RichTextBody',
    'axios.com': 'DraftjsBlocks_draftjs__fm3S2',
    'foxnews.com':'article-content'}


    # Add more websites and their corresponding selectors as needed
def scrape_actual_news(url_news, domain):
    # domain = url.split('/')[2]
    
    # # Check if the domain is in the selectors dictionary
    # if domain in selectors:
        # Send a GET request to the URL
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url_news, headers=headers)
    print(response.status_code)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser') 

        selector = selectors[domain]   

        # Extract title and content using the specified selectors
        article_body = soup.find('div', class_=selector).text.strip()
        # Return the extracted content
        return {
            'Content': article_body
        }
    else:
        print(f"Failed to retrieve content from {url_news}. Status code: {response.status_code}")
        return response.status_code

200
{'Content': 'close      Video Trump reportedly promotes abortion ban after 16 weeks of pregnancy Former President Donald Trump discusses ‘common sense’ on ‘MediaBuzz,’ and says ‘you have to have the three exceptions’ on abortions. Join Fox News for access to this content Plus special access to select articles and other premium content with your account - free of charge. Please enter a valid email address.   By entering your email and pushing continue, you are agreeing to Fox News\' Terms of Use and Privacy Policy, which includes our Notice of Financial Incentive. To access the content, check your email and follow the instructions provided.\n       Having trouble? Click here. The Supreme Court appeared ready to dismiss a challenge on Tuesday from opponents of the Food and Drug Administration\'s (FDA) relaxed requirements for dispensing an abortion drug, used to terminate nearly two-thirds of pregnancies in the country. That would be a victory for the Biden administration and abortio

In [112]:
def scrape_url_allsides(url_news):

    response = requests.get(url_news)
    soup = BeautifulSoup(response.content, 'html.parser')
    read_more_div = soup.find('div', class_='read-more-story')
    if read_more_div!=None:
        full_story_url = read_more_div.find('a')['href']
        return full_story_url
    else:
        return None

In [146]:
def scrape_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extract data from the page using BeautifulSoup
    # Example: titles = soup.find_all('h2', class_='title')
    articles = soup.find_all('div', class_='views-row')
    data = []
    for article in articles:
        published_date = article.find('p', class_='search-result-publish-date').text.strip() if article.find('p', class_='search-result-publish-date') else 'NaN'
        title = article.find('h3', class_='search-result-title').text.strip() if article.find('h3', class_='search-result-title') else 'NaN'
        content = article.find('a', class_='search-result-body').text.strip() if article.find('a', class_='search-result-body') else 'NaN'
        url = article.find('a', class_='search-result-body')['href'] if article.find('a', class_='search-result-body') else 'NaN'
        news_channel = article.find('a', class_='search-result-source').text.strip() if article.find('a', class_='search-result-source') else 'NaN'
        bias = article.find('img')['src'].split('-')[-1].strip().split('.')[0] if article.find('img') else 'NaN'
        tags_element = article.find('p', class_='search-result-tags') if article.find('p', class_='search-result-tags') else None
        if tags_element:
            tags_html = tags_element.find('span', class_='field-content').prettify()  # Get the HTML content of the tags
            soup = BeautifulSoup(tags_html, 'html.parser')  # Create a new BeautifulSoup object with the HTML content
            tags = soup.find_all('a')  # Find all <a> tags within the HTML
            tag_texts = [tag.text for tag in tags]  # Extract tag text from <a> tags

        else:
            tag_texts = None

        news_channel_url = scrape_url_allsides(url) if url != 'NaN' else 'NaN'
        
        data.append({
            'Published Date': published_date,
            'Title': title,
            'Content': content,
            'URL': url,
            'News Channel': news_channel,
            "News Channel URL": news_channel_url,
            'Bias': bias,
            'Tags':tag_texts,
        })
    
    return data

scrape_page(url)


<p class="search-result-tags"><span class="field-content"><a href="/tags/supreme-court">Supreme Court</a>, <a href="/tags/abortion">Abortion</a>, <a href="/tags/abortion-pills">Abortion Pills</a>, <a href="/tags/healthcare">Healthcare</a>, <a href="/tags/womens-issues">Women's Issues</a>, <a href="/tags/pharmaceuticals">Pharmaceuticals</a></span></p>


TypeError: 'NoneType' object is not callable

In [132]:
def return_all_pages(n,url):
    data = scrape_page(url)
    for i in range(1,n):
        url_new = url + "&page=" + str(i)
        data += scrape_page(url_new)

    return data

abortion_data = return_all_pages(10,url)



In [133]:
df = pd.DataFrame(abortion_data)

print(df.head(10))


  Published Date                                              Title  \
0    Mar 26 2024  Abortion pill ruling could be "catastrophic" f...   
1    Mar 26 2024  Supreme Court seems likely to preserve access ...   
2    Mar 26 2024  Supreme Court Appears Unlikely to Upend Aborti...   
3    Mar 26 2024  Supreme Court appears inclined to preserve bro...   
4    Mar 26 2024  We’re who Justice Alito said should make abort...   
5    Mar 26 2024       Protests rage amid SCOTUS abortion arguments   
6    Mar 26 2024  Florida arrests petition gatherer accused of s...   
7    Mar 26 2024  Today on Texas Standard: An Indigenous perspec...   
8    Mar 26 2024  Abortion pill usage surged post-Roe. These num...   
9    Mar 26 2024  Supreme Court considers abortion pill access a...   

                                             Content  \
0  The Supreme Court today will hear a challenge ...   
1  The Supreme Court on Tuesday seemed likely to ...   
2  The Supreme Court appeared likely to preserve .

In [134]:
df.to_csv('output.csv', index=False)


In [131]:
df["News Channel"].value_counts

<bound method IndexOpsMixin.value_counts of 0                            Axios
1                 Associated Press
2       Wall Street Journal (News)
3           Fox News (Online News)
4     Multiple Writers - Lean Left
5                Fox 4 Kansas City
6                 Florida Politics
7                   Texas Standard
8                Los Angeles Times
9                NBC News (Online)
10               CBS News (Online)
11                           Axios
12           New York Times (News)
13               Fox 4 Kansas City
14             Florida Times-Union
15                       Patch.com
16            Catholic News Agency
17                 Washington Post
18                            KFGO
19                          WCAX 3
20                      Dan Schnur
21                       Patch.com
22                        Politico
23         "USA Today" Contributor
24                       The State
25                     ABC 11 WTVD
26                  Breitbart News
27         

In [ ]:
# should we use article headlines or the actual content of the news
# what should be the number of pages to be parsed through 
# what should be the number of topics to be considered?